In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
%matplotlib inline


We've talked about Random Forests. Now it's time to build one.

Here we'll use data from Lending Club to predict the state of a loan given some information about it. You can find the dataset [here](https://www.lendingclub.com/info/download-data.action). We'll use 2015 data. ([Thinkful mirror](https://www.dropbox.com/s/m7z42lubaiory33/LoanStats3d.csv?dl=0))

In [2]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    'LoanStats3d.csv',
    skipinitialspace=True,
    header=1
)


c:\users\mango\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [3]:
means = y2015[['loan_amnt','funded_amnt','funded_amnt_inv', 'installment']].mean(axis=0)
stds = y2015[['loan_amnt','funded_amnt','funded_amnt_inv', 'installment']].std(axis=0)
y2015['amount'] = (((y2015[['loan_amnt','funded_amnt','funded_amnt_inv', 'installment']] - means) / stds).mean(axis=1))

y2015 = y2015.drop(['loan_amnt', 'funded_amnt','member_id', 'url',
                    'installment', 'funded_amnt_inv', 'total_pymnt',
                    'settlement_amount', 'annual_inc_joint',
                    'hardship_payoff_balance_amount', 'open_acc', 'tax_liens', 'hardship_amount',
                    'orig_projected_additional_accrued_interest', 'total_rec_int', 'dti_joint'], 1)


In [5]:
y2015.corr()

,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,pub_rec,revol_bal,total_acc,out_prncp,...,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,deferral_term,hardship_length,hardship_dpd,hardship_last_payment_amount,settlement_percentage,settlement_term,amount
annual_inc,1.000000,-0.174934,0.037910,0.022990,-0.043663,-0.087538,-0.006739,0.270174,0.159804,0.053059,...,NaN,NaN,NaN,NaN,NaN,0.008168,0.212510,-0.001744,0.022872,0.307073
dti,-0.174934,1.000000,-0.019319,0.006059,0.005077,0.083465,-0.059443,0.129258,0.217357,0.063361,...,NaN,NaN,NaN,NaN,NaN,0.000124,0.074712,0.018289,0.043506,0.015218
delinq_2yrs,0.037910,-0.019319,1.000000,0.036139,-0.553754,-0.095502,-0.015220,-0.035801,0.114623,0.003996,...,NaN,NaN,NaN,NaN,NaN,0.032806,-0.010438,-0.003636,-0.013766,-0.008449
inq_last_6mths,0.022990,0.006059,0.036139,1.000000,0.026783,-0.070857,0.097382,-0.012303,0.161029,-0.037180,...,NaN,NaN,NaN,NaN,NaN,0.012744,0.000816,0.012859,-0.009267,-0.031210
mths_since_last_delinq,-0.043663,0.005077,-0.553754,0.026783,1.000000,-0.007374,0.103950,-0.027622,-0.033478,-0.022546,...,NaN,NaN,NaN,NaN,NaN,-0.023489,-0.009877,0.001483,0.013671,-0.035793
mths_since_last_record,-0.087538,0.083465,-0.095502,-0.070857,-0.007374,1.000000,-0.265775,-0.022070,-0.128978,-0.016958,...,NaN,NaN,NaN,NaN,NaN,-0.093132,-0.017284,-0.016336,-0.017441,-0.017031
pub_rec,-0.006739,-0.059443,-0.015220,0.097382,0.103950,-0.265775,1.000000,-0.106714,0.014886,-0.031497,...,NaN,NaN,NaN,NaN,NaN,0.053232,-0.033093,-0.007328,-0.004052,-0.085968
revol_bal,0.270174,0.129258,-0.035801,-0.012303,-0.027622,-0.022070,-0.106714,1.000000,0.180191,0.094051,...,NaN,NaN,NaN,NaN,NaN,0.006802,0.131775,-0.001489,0.054971,0.333209
total_acc,0.159804,0.217357,0.114623,0.161029,-0.033478,-0.128978,0.014886,0.180191,1.000000,0.028232,...,NaN,NaN,NaN,NaN,NaN,-0.032408,0.084087,0.004425,0.043725,0.204474
out_prncp,0.053059,0.063361,0.003996,-0.037180,-0.022546,-0.016958,-0.031497,0.094051,0.028232,1.000000,...,NaN,NaN,NaN,NaN,NaN,0.014848,0.258048,-0.015115,0.028018,0.231399


In [4]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()

y2015 = y2015[:-2]

X = y2015.drop('loan_status', 1)
X = X.dropna(axis=1)
Y = y2015['loan_status']
Y = Y.fillna('')
X = pd.get_dummies(X)


cross_val_score(rfc, X, Y, cv=10)


c:\users\mango\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
c:\users\mango\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\mango\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\mango\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10

KeyboardInterrupt: 

In [ ]:
sklearn_pca = PCA(n_components=4)
Y_sklearn = sklearn_pca.fit_transform(X)

cross_val_score(rfc, X, Y, cv=10)